Importa as bibliotecas de funcionalidades que usaremos.


In [11]:
import pandas as pd
import glob
from datetime import date
import numpy as np

Importa as planilhas com os dados de aeródromos brasileiros para DataFrames. 


In [12]:
aero_publicos=pd.read_excel('BasesCenipa/aerodromospublicos-12.xls', skiprows=2)
aero_privados=pd.read_excel('BasesCenipa/aerodromosprivados-9.xls')

1.   Mescla em um único DataFrame as duas planilhas importadas.
2.   Cria uma lista de código OACI de todos os aeroportos do Brasil.
3.   Verifica o tipo do dado de cada item da lista.
4.   Deleta aquele item que for um número inteiro.
5.   Procura pelo item 'SNRA*'
6.   Substitui o item acima por 'SNRA'

Os itens 3 a 6 foram criados para corrigir erros da planilha excel importada.



In [13]:
df_aero_pub_e_priv=pd.merge(aero_publicos, aero_privados, how='outer').drop_duplicates(subset='CÓDIGO OACI') # Tira duplicado com base na coluna 'CÓDIGO
aero_pub_e_priv=list(df_aero_pub_e_priv['CÓDIGO OACI'])
ls= [type(item) for item in aero_pub_e_priv]
aero_pub_e_priv.pop(ls.index(int))
snra=aero_pub_e_priv.index('SNRA*')
aero_pub_e_priv[snra]='SNRA'

Cria lista de código OACI de aeroportos públicos. Roda a rotina para verificar se existe algum número inteiro e dropa se tiver.

In [14]:
lista_aero_oaci_publico=list(aero_publicos['CÓDIGO OACI'])
try:
  ls= [type(item) for item in lista_aero_oaci_publico]
  lista_aero_oaci_publico.pop(ls.index(int))
except:
  None

Cria lista de código OACI de aeroportos privados. Roda a rotina para verificar se existe algum número inteiro e dropa se tiver.

In [15]:
lista_aero_oaci_privado=list(aero_privados['CÓDIGO OACI'])
try:
  ls= [type(item) for item in lista_aero_oaci_privado]
  lista_aero_oaci_privado.pop(ls.index(int))
except:
  None
lista_aero_oaci_privado=sorted(lista_aero_oaci_privado)

Cria uma data final pelo dia de hoje.


In [16]:
data=date.today()
dia=data.strftime('%d')
mes=data.strftime('%m')
ano=data.year
datafinal=str(dia)+"%2F"+str(mes)+"%2F"+str(ano)

Lista os aeroportos já baixados e ajusta a lista para aeroportos que faltam baixar. Foi criado por culpa de falha na hora de fazer download de dados de alguns aeroportos com muitas informações.


In [17]:
def para_baixar():
    global aeroportos_para_baixar
    aeroportos_na_pasta=glob.glob('BasesCenipa/ficheiro *.csv')
    aeroportos_ja_baixados=[]
    for item in aeroportos_na_pasta: aeroportos_ja_baixados.append(str(item[21:25]))
    aeroportos_para_baixar=list(np.setdiff1d(aero_pub_e_priv, aeroportos_ja_baixados))

Primeira etapa do download dos arquivos do Cenipa. Tenta baixar todos os dados daquele aeroporto, sem definir o ano. Gera uma lista daqueles aeroportos não baixados para segunda etapa de download.

PRECISA CORRIGIR A FALHA DO "FALTAM XXX DE XXX PARA TENTAR BAIXAR". Não está reduzindo a medida que baixa.

In [18]:
nao_baixados=[]
ficheiro={}
para_baixar()
for aeroporto in aeroportos_para_baixar:
  try:
    url=f'http://sistema.cenipa.aer.mil.br/cenipa/sigra/export_excel?sigra=pesquisa&identificacao=&matricula=&ano_pesquisa=&data_inicial=01%2F01%2F2011&data_final={datafinal}&ICAO={aeroporto}&tipoReporte=&classificacao_ocorrencia=&Parte_da_aeronave=&area_seguranca=&Especie=&aviacaoTipo=&Danos_Prejuizos=&anvOperador=&Efeito_no_voo=&codicoes_ceu=&tripulacao_alertada=&Fase_do_Voo=&parte_dia=&precipitacao=&funcao=&pg=1&Submit=Executar+pesquisa'
    ler_arquivo=pd.read_html(url)
    ficheiro[aeroporto]=pd.DataFrame(ler_arquivo[0]).droplevel(1,axis=1).fillna(0)
    ficheiro[aeroporto].to_csv(f'BasesCenipa/ficheiro {aeroporto}.csv')
    print(f'O arquivo do aeroporto {aeroporto} foi criado.')
  except:
    print(f'Não foi possível importar do aeroporto {aeroporto}.')
    if aeroporto not in nao_baixados:
      nao_baixados.append(aeroporto)
  finally:
    para_baixar()
    print(f'Faltam {len(aeroportos_para_baixar)} de {len(aero_pub_e_priv)} para tentar baixar. \nNão foi possível baixar dados de {len(nao_baixados)} aeroportos.')
else:
  print("Os aeroportos abaixo serão baixados de ano em ano: ", nao_baixados, )

Não foi possível importar do aeroporto SBBR.
Faltam 2658 de 2737 para tentar baixar. 
Não foi possível baixar dados de 1 aeroportos.
Não foi possível importar do aeroporto SBCG.
Faltam 2658 de 2737 para tentar baixar. 
Não foi possível baixar dados de 2 aeroportos.
Não foi possível importar do aeroporto SBCP.
Faltam 2658 de 2737 para tentar baixar. 
Não foi possível baixar dados de 3 aeroportos.
Não foi possível importar do aeroporto SBCT.
Faltam 2658 de 2737 para tentar baixar. 
Não foi possível baixar dados de 4 aeroportos.
Não foi possível importar do aeroporto SBGL.
Faltam 2658 de 2737 para tentar baixar. 
Não foi possível baixar dados de 5 aeroportos.
Não foi possível importar do aeroporto SBGR.
Faltam 2658 de 2737 para tentar baixar. 
Não foi possível baixar dados de 6 aeroportos.
Não foi possível importar do aeroporto SBKP.
Faltam 2658 de 2737 para tentar baixar. 
Não foi possível baixar dados de 7 aeroportos.
O arquivo do aeroporto SBNF foi criado.
Faltam 2657 de 2737 para tent

O arquivo do aeroporto SDAG foi criado.
Faltam 2604 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDAI foi criado.
Faltam 2603 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDAJ foi criado.
Faltam 2602 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDAM foi criado.
Faltam 2601 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDAN foi criado.
Faltam 2600 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDAO foi criado.
Faltam 2599 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDAR foi criado.
Faltam 2598 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDAS foi criado.
Faltam 2597 de 2737 para tentar baixar. 
Não foi possível

O arquivo do aeroporto SDEV foi criado.
Faltam 2540 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDFC foi criado.
Faltam 2539 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDFD foi criado.
Faltam 2538 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDFE foi criado.
Faltam 2537 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDFF foi criado.
Faltam 2536 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDFH foi criado.
Faltam 2535 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDFI foi criado.
Faltam 2534 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDFK foi criado.
Faltam 2533 de 2737 para tentar baixar. 
Não foi possível

O arquivo do aeroporto SDKJ foi criado.
Faltam 2476 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDKK foi criado.
Faltam 2475 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDKL foi criado.
Faltam 2474 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDKO foi criado.
Faltam 2473 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDKP foi criado.
Faltam 2472 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDKQ foi criado.
Faltam 2471 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDKX foi criado.
Faltam 2470 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDKZ foi criado.
Faltam 2469 de 2737 para tentar baixar. 
Não foi possível

O arquivo do aeroporto SDOL foi criado.
Faltam 2412 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDOQ foi criado.
Faltam 2411 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDOR foi criado.
Faltam 2410 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDOS foi criado.
Faltam 2409 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDOU foi criado.
Faltam 2408 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDOV foi criado.
Faltam 2407 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDOW foi criado.
Faltam 2406 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SDOX foi criado.
Faltam 2405 de 2737 para tentar baixar. 
Não foi possível

...

O arquivo do aeroporto SWJW foi criado.
Faltam 290 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWJX foi criado.
Faltam 289 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWJY foi criado.
Faltam 288 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWJZ foi criado.
Faltam 287 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWKA foi criado.
Faltam 286 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWKB foi criado.
Faltam 285 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWKC foi criado.
Faltam 284 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWKE foi criado.
Faltam 283 de 2737 para tentar baixar. 
Não foi possível baixar 

O arquivo do aeroporto SWNK foi criado.
Faltam 225 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWNL foi criado.
Faltam 224 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWNN foi criado.
Faltam 223 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWNQ foi criado.
Faltam 222 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWNR foi criado.
Faltam 221 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWNS foi criado.
Faltam 220 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWNU foi criado.
Faltam 219 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWOA foi criado.
Faltam 218 de 2737 para tentar baixar. 
Não foi possível baixar 

O arquivo do aeroporto SWRI foi criado.
Faltam 160 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWRL foi criado.
Faltam 159 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWRM foi criado.
Faltam 158 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWRO foi criado.
Faltam 157 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWRP foi criado.
Faltam 156 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWRQ foi criado.
Faltam 155 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWRS foi criado.
Faltam 154 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWRT foi criado.
Faltam 153 de 2737 para tentar baixar. 
Não foi possível baixar 

O arquivo do aeroporto SWVI foi criado.
Faltam 95 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWVJ foi criado.
Faltam 94 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWVK foi criado.
Faltam 93 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWVN foi criado.
Faltam 92 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWVR foi criado.
Faltam 91 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWVS foi criado.
Faltam 90 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWVU foi criado.
Faltam 89 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWVV foi criado.
Faltam 88 de 2737 para tentar baixar. 
Não foi possível baixar dados de

O arquivo do aeroporto SWYW foi criado.
Faltam 30 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWYX foi criado.
Faltam 29 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWYY foi criado.
Faltam 28 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWZA foi criado.
Faltam 27 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWZC foi criado.
Faltam 26 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWZD foi criado.
Faltam 25 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWZE foi criado.
Faltam 24 de 2737 para tentar baixar. 
Não foi possível baixar dados de 11 aeroportos.
O arquivo do aeroporto SWZG foi criado.
Faltam 23 de 2737 para tentar baixar. 
Não foi possível baixar dados de

Com base na lista de aeroportos não baixados na primeira etapa, gera um dicionário onde a chave é o aeroporto e os valores são os anos para baixar.

In [19]:
anos = ['2011','2012','2013','2014','2015','2016','2017','2018','2019','2020']
aeroporto_ano_para_baixar={}
for aeroporto in nao_baixados:
  aeroporto_ano_para_baixar[aeroporto]=anos
aeroporto_ano_para_baixar
print(aeroporto_ano_para_baixar)

{'SBBR': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBCG': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBCP': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBCT': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBGL': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBGR': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBKP': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBPA': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBRF': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBSP': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBSV': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']}


Código abaixo é para identificar aqueles anos já baixados dos dados que não foram baixados na primeira etapa de download. 
Cria uma dicionário onde a key é o aeroporto onde precisa baixar algum ano e os valores são os anos a serem baixados.
Compara um dicionário de dados por ano já baixados com aqueles para baixar criados na primeira etapa.


In [20]:
aeroporto_ano_na_pasta=glob.glob('BasesCenipa/ficheiro_*_*.csv')
lista_aeroporto_ano_ja_baixado=[]
dict_aeroporto_ano_ja_baixado={}
aeroportos_anos_para_baixar={}
for arquivo in aeroporto_ano_na_pasta:
  lista_aeroporto_ano_ja_baixado.append(str(arquivo[21:25])) if str(arquivo[21:25]) not in lista_aeroporto_ano_ja_baixado else None
for oaci in lista_aeroporto_ano_ja_baixado:
  aeroporto_ano_na_pasta=glob.glob(f'BasesCenipa/ficheiro_{oaci}*.csv')
  for arquivo_baixado_ano in aeroporto_ano_na_pasta:
    if dict_aeroporto_ano_ja_baixado.get(oaci) == None:
      dict_aeroporto_ano_ja_baixado[oaci]=[str(arquivo_baixado_ano[26:30])]
    else:
      dict_aeroporto_ano_ja_baixado[oaci].append(str(arquivo_baixado_ano[26:30]))
for oaci in nao_baixados:
  if dict_aeroporto_ano_ja_baixado.get(oaci) == None:
    aeroportos_anos_para_baixar[oaci]=anos
  else:
    if len(dict_aeroporto_ano_ja_baixado[oaci]) < 10:
      aeroportos_anos_para_baixar[oaci]=list(np.setdiff1d(aeroporto_ano_para_baixar[oaci], dict_aeroporto_ano_ja_baixado[oaci]))

In [21]:
print(dict_aeroporto_ano_ja_baixado)
print(aeroporto_ano_para_baixar)
print(aeroportos_anos_para_baixar)

{}
{'SBBR': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBCG': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBCP': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBCT': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBGL': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBGR': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBKP': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBPA': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBRF': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBSP': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'SBSV': ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']}
{'SBBR

Segunda etapa de download - após verificar quais são os aeroportos para baixar e os anos que sobraram para baixar, baixa aqueles anos que faltam para cada aeroporto. 

Cria uma dicionario de anos por aeroporto que não foram baixados.

In [22]:
ano_nao_baixado={}
for aeroporto in list(aeroportos_anos_para_baixar.keys()):
  for ano in aeroportos_anos_para_baixar[aeroporto]:
    try:
      url=f'http://sistema.cenipa.aer.mil.br/cenipa/sigra/export_excel?sigra=pesquisa&identificacao=&matricula=&ano_pesquisa={ano}&data_inicial=&data_final=&ICAO={aeroporto}&tipoReporte=&classificacao_ocorrencia=&Parte_da_aeronave=&area_seguranca=&Especie=&aviacaoTipo=&Danos_Prejuizos=&anvOperador=&Efeito_no_voo=&codicoes_ceu=&tripulacao_alertada=&Fase_do_Voo=&parte_dia=&precipitacao=&funcao=&pg=1&Submit=Executar+pesquisa'
      ler_arquivo=pd.read_html(url)
      ler_arquivo=pd.DataFrame(ler_arquivo[0]).droplevel(1,axis=1).fillna(0)
      ler_arquivo.to_csv(f'BasesCenipa/ficheiro_{aeroporto}_{ano}.csv')
      print(f'O arquivo do aeroporto {aeroporto} para ano de {ano} foi criado. \n')
    except:
      print(f'Não foi possível importar do aeroporto {aeroporto} ano {ano}. \n')
      if ano_nao_baixado.get(aeroporto) == None:
        ano_nao_baixado[aeroporto]=[ano]
      else:
        ano_nao_baixado[aeroporto].append([ano])
print(ano_nao_baixado)

O arquivo do aeroporto SBBR para ano de 2011 foi criado. 

O arquivo do aeroporto SBBR para ano de 2012 foi criado. 

O arquivo do aeroporto SBBR para ano de 2013 foi criado. 

O arquivo do aeroporto SBBR para ano de 2014 foi criado. 

O arquivo do aeroporto SBBR para ano de 2015 foi criado. 

Não foi possível importar do aeroporto SBBR ano 2016. 

O arquivo do aeroporto SBBR para ano de 2017 foi criado. 

O arquivo do aeroporto SBBR para ano de 2018 foi criado. 

O arquivo do aeroporto SBBR para ano de 2019 foi criado. 

O arquivo do aeroporto SBBR para ano de 2020 foi criado. 

O arquivo do aeroporto SBCG para ano de 2011 foi criado. 

O arquivo do aeroporto SBCG para ano de 2012 foi criado. 

O arquivo do aeroporto SBCG para ano de 2013 foi criado. 

O arquivo do aeroporto SBCG para ano de 2014 foi criado. 

O arquivo do aeroporto SBCG para ano de 2015 foi criado. 

O arquivo do aeroporto SBCG para ano de 2016 foi criado. 

O arquivo do aeroporto SBCG para ano de 2017 foi criado. 



In [23]:
for aeroporto in ano_nao_baixado:
  for ano in ano_nao_baixado[aeroporto]:
    url=f'http://sistema.cenipa.aer.mil.br/cenipa/sigra/export_excel?sigra=pesquisa&identificacao=&matricula=&ano_pesquisa=&data_inicial=01%2F01%2F{ano}&data_final=30%2F06%2F{ano}&ICAO={aeroporto}&tipoReporte=&classificacao_ocorrencia=&Parte_da_aeronave=&area_seguranca=&Especie=&aviacaoTipo=&Danos_Prejuizos=&anvOperador=&Efeito_no_voo=&codicoes_ceu=&tripulacao_alertada=&Fase_do_Voo=&parte_dia=&precipitacao=&funcao=&pg=1&Submit=Executar+pesquisa'
    url1=f'http://sistema.cenipa.aer.mil.br/cenipa/sigra/export_excel?sigra=pesquisa&identificacao=&matricula=&ano_pesquisa=&data_inicial=01%2F07%2F{ano}&data_final=31%2F12%2F{ano}&ICAO={aeroporto}&tipoReporte=&classificacao_ocorrencia=&Parte_da_aeronave=&area_seguranca=&Especie=&aviacaoTipo=&Danos_Prejuizos=&anvOperador=&Efeito_no_voo=&codicoes_ceu=&tripulacao_alertada=&Fase_do_Voo=&parte_dia=&precipitacao=&funcao=&pg=1&Submit=Executar+pesquisa'
    ler_arquivo=pd.read_html(url)
    ler_arquivo=pd.DataFrame(ler_arquivo[0]).droplevel(1,axis=1).fillna(0)
    ler_arquivo.to_csv(f'BasesCenipa/ficheiro_{aeroporto}_{ano}_1º.csv')
    print(f'O arquivo {aeroporto} ano {ano} 1º Semestre foi baixado!')
    ler_arquivo=pd.read_html(url1)
    ler_arquivo=pd.DataFrame(ler_arquivo[0]).droplevel(1,axis=1).fillna(0)
    ler_arquivo.to_csv(f'BasesCenipa/ficheiro_{aeroporto}_{ano}_2º.csv')
    print(f'O arquivo {aeroporto} ano {ano} 2º Semestre foi baixado!')
    

O arquivo SBBR ano 2016 1º Semestre foi baixado!
O arquivo SBBR ano 2016 2º Semestre foi baixado!
O arquivo SBGL ano 2019 1º Semestre foi baixado!
O arquivo SBGL ano 2019 2º Semestre foi baixado!


In [24]:
arquivos= glob.glob("BasesCenipa/ficheiro*.csv")

In [25]:
base={}
n=1
colunas = list(range(1,42,1))
for arquivo in arquivos:
  ler_arquivo=pd.read_csv(arquivo, usecols=colunas)
  print(f'Arquivo {arquivo} lido.')
  n=n+1
  base[str(n)]=ler_arquivo

Arquivo BasesCenipa\ficheiro SBAA.csv lido.
Arquivo BasesCenipa\ficheiro SBAC.csv lido.
Arquivo BasesCenipa\ficheiro SBAE.csv lido.
Arquivo BasesCenipa\ficheiro SBAQ.csv lido.
Arquivo BasesCenipa\ficheiro SBAR.csv lido.
Arquivo BasesCenipa\ficheiro SBAT.csv lido.
Arquivo BasesCenipa\ficheiro SBAU.csv lido.
Arquivo BasesCenipa\ficheiro SBAX.csv lido.
Arquivo BasesCenipa\ficheiro SBBE.csv lido.
Arquivo BasesCenipa\ficheiro SBBG.csv lido.
Arquivo BasesCenipa\ficheiro SBBH.csv lido.
Arquivo BasesCenipa\ficheiro SBBI.csv lido.
Arquivo BasesCenipa\ficheiro SBBP.csv lido.
Arquivo BasesCenipa\ficheiro SBBU.csv lido.
Arquivo BasesCenipa\ficheiro SBBV.csv lido.
Arquivo BasesCenipa\ficheiro SBBW.csv lido.
Arquivo BasesCenipa\ficheiro SBCA.csv lido.
Arquivo BasesCenipa\ficheiro SBCB.csv lido.
Arquivo BasesCenipa\ficheiro SBCD.csv lido.
Arquivo BasesCenipa\ficheiro SBCF.csv lido.
Arquivo BasesCenipa\ficheiro SBCH.csv lido.
Arquivo BasesCenipa\ficheiro SBCI.csv lido.
Arquivo BasesCenipa\ficheiro SBC

Arquivo BasesCenipa\ficheiro SDEN.csv lido.
Arquivo BasesCenipa\ficheiro SDEO.csv lido.
Arquivo BasesCenipa\ficheiro SDEP.csv lido.
Arquivo BasesCenipa\ficheiro SDER.csv lido.
Arquivo BasesCenipa\ficheiro SDES.csv lido.
Arquivo BasesCenipa\ficheiro SDET.csv lido.
Arquivo BasesCenipa\ficheiro SDEV.csv lido.
Arquivo BasesCenipa\ficheiro SDFC.csv lido.
Arquivo BasesCenipa\ficheiro SDFD.csv lido.
Arquivo BasesCenipa\ficheiro SDFE.csv lido.
Arquivo BasesCenipa\ficheiro SDFF.csv lido.
Arquivo BasesCenipa\ficheiro SDFH.csv lido.
Arquivo BasesCenipa\ficheiro SDFI.csv lido.
Arquivo BasesCenipa\ficheiro SDFK.csv lido.
Arquivo BasesCenipa\ficheiro SDFT.csv lido.
Arquivo BasesCenipa\ficheiro SDFU.csv lido.
Arquivo BasesCenipa\ficheiro SDFX.csv lido.
Arquivo BasesCenipa\ficheiro SDFY.csv lido.
Arquivo BasesCenipa\ficheiro SDFZ.csv lido.
Arquivo BasesCenipa\ficheiro SDGA.csv lido.
Arquivo BasesCenipa\ficheiro SDGB.csv lido.
Arquivo BasesCenipa\ficheiro SDGC.csv lido.
Arquivo BasesCenipa\ficheiro SDG

Arquivo BasesCenipa\ficheiro SDSH.csv lido.
Arquivo BasesCenipa\ficheiro SDSI.csv lido.
Arquivo BasesCenipa\ficheiro SDSJ.csv lido.
Arquivo BasesCenipa\ficheiro SDSK.csv lido.
Arquivo BasesCenipa\ficheiro SDSL.csv lido.
Arquivo BasesCenipa\ficheiro SDSM.csv lido.
Arquivo BasesCenipa\ficheiro SDSP.csv lido.
Arquivo BasesCenipa\ficheiro SDSQ.csv lido.
Arquivo BasesCenipa\ficheiro SDST.csv lido.
Arquivo BasesCenipa\ficheiro SDSV.csv lido.
Arquivo BasesCenipa\ficheiro SDTA.csv lido.
Arquivo BasesCenipa\ficheiro SDTB.csv lido.
Arquivo BasesCenipa\ficheiro SDTD.csv lido.
Arquivo BasesCenipa\ficheiro SDTE.csv lido.
Arquivo BasesCenipa\ficheiro SDTF.csv lido.
Arquivo BasesCenipa\ficheiro SDTH.csv lido.
Arquivo BasesCenipa\ficheiro SDTI.csv lido.
Arquivo BasesCenipa\ficheiro SDTK.csv lido.
Arquivo BasesCenipa\ficheiro SDTO.csv lido.
Arquivo BasesCenipa\ficheiro SDTP.csv lido.
Arquivo BasesCenipa\ficheiro SDTR.csv lido.
Arquivo BasesCenipa\ficheiro SDTS.csv lido.
Arquivo BasesCenipa\ficheiro SDT

Arquivo BasesCenipa\ficheiro SIEQ.csv lido.
Arquivo BasesCenipa\ficheiro SIEV.csv lido.
Arquivo BasesCenipa\ficheiro SIEW.csv lido.
Arquivo BasesCenipa\ficheiro SIEX.csv lido.
Arquivo BasesCenipa\ficheiro SIEZ.csv lido.
Arquivo BasesCenipa\ficheiro SIFA.csv lido.
Arquivo BasesCenipa\ficheiro SIFC.csv lido.
Arquivo BasesCenipa\ficheiro SIFD.csv lido.
Arquivo BasesCenipa\ficheiro SIFE.csv lido.
Arquivo BasesCenipa\ficheiro SIFF.csv lido.
Arquivo BasesCenipa\ficheiro SIFJ.csv lido.
Arquivo BasesCenipa\ficheiro SIFM.csv lido.
Arquivo BasesCenipa\ficheiro SIFN.csv lido.
Arquivo BasesCenipa\ficheiro SIFO.csv lido.
Arquivo BasesCenipa\ficheiro SIFQ.csv lido.
Arquivo BasesCenipa\ficheiro SIFS.csv lido.
Arquivo BasesCenipa\ficheiro SIFT.csv lido.
Arquivo BasesCenipa\ficheiro SIFU.csv lido.
Arquivo BasesCenipa\ficheiro SIFV.csv lido.
Arquivo BasesCenipa\ficheiro SIFW.csv lido.
Arquivo BasesCenipa\ficheiro SIFX.csv lido.
Arquivo BasesCenipa\ficheiro SIGA.csv lido.
Arquivo BasesCenipa\ficheiro SIG

Arquivo BasesCenipa\ficheiro SISS.csv lido.
Arquivo BasesCenipa\ficheiro SISU.csv lido.
Arquivo BasesCenipa\ficheiro SISV.csv lido.
Arquivo BasesCenipa\ficheiro SISW.csv lido.
Arquivo BasesCenipa\ficheiro SISX.csv lido.
Arquivo BasesCenipa\ficheiro SISY.csv lido.
Arquivo BasesCenipa\ficheiro SISZ.csv lido.
Arquivo BasesCenipa\ficheiro SITA.csv lido.
Arquivo BasesCenipa\ficheiro SITB.csv lido.
Arquivo BasesCenipa\ficheiro SITC.csv lido.
Arquivo BasesCenipa\ficheiro SITF.csv lido.
Arquivo BasesCenipa\ficheiro SITH.csv lido.
Arquivo BasesCenipa\ficheiro SITI.csv lido.
Arquivo BasesCenipa\ficheiro SITJ.csv lido.
Arquivo BasesCenipa\ficheiro SITL.csv lido.
Arquivo BasesCenipa\ficheiro SITM.csv lido.
Arquivo BasesCenipa\ficheiro SITR.csv lido.
Arquivo BasesCenipa\ficheiro SITS.csv lido.
Arquivo BasesCenipa\ficheiro SITT.csv lido.
Arquivo BasesCenipa\ficheiro SITU.csv lido.
Arquivo BasesCenipa\ficheiro SITV.csv lido.
Arquivo BasesCenipa\ficheiro SITX.csv lido.
Arquivo BasesCenipa\ficheiro SIU

...

Arquivo BasesCenipa\ficheiro SSLR.csv lido.
Arquivo BasesCenipa\ficheiro SSLS.csv lido.
Arquivo BasesCenipa\ficheiro SSLT.csv lido.
Arquivo BasesCenipa\ficheiro SSLW.csv lido.
Arquivo BasesCenipa\ficheiro SSLX.csv lido.
Arquivo BasesCenipa\ficheiro SSMB.csv lido.
Arquivo BasesCenipa\ficheiro SSMC.csv lido.
Arquivo BasesCenipa\ficheiro SSME.csv lido.
Arquivo BasesCenipa\ficheiro SSMF.csv lido.
Arquivo BasesCenipa\ficheiro SSMH.csv lido.
Arquivo BasesCenipa\ficheiro SSMI.csv lido.
Arquivo BasesCenipa\ficheiro SSMJ.csv lido.
Arquivo BasesCenipa\ficheiro SSMK.csv lido.
Arquivo BasesCenipa\ficheiro SSML.csv lido.
Arquivo BasesCenipa\ficheiro SSMP.csv lido.
Arquivo BasesCenipa\ficheiro SSMR.csv lido.
Arquivo BasesCenipa\ficheiro SSMT.csv lido.
Arquivo BasesCenipa\ficheiro SSMV.csv lido.
Arquivo BasesCenipa\ficheiro SSMW.csv lido.
Arquivo BasesCenipa\ficheiro SSMX.csv lido.
Arquivo BasesCenipa\ficheiro SSMY.csv lido.
Arquivo BasesCenipa\ficheiro SSMZ.csv lido.
Arquivo BasesCenipa\ficheiro SSN

Arquivo BasesCenipa\ficheiro SSWZ.csv lido.
Arquivo BasesCenipa\ficheiro SSXA.csv lido.
Arquivo BasesCenipa\ficheiro SSXD.csv lido.
Arquivo BasesCenipa\ficheiro SSXF.csv lido.
Arquivo BasesCenipa\ficheiro SSXG.csv lido.
Arquivo BasesCenipa\ficheiro SSXH.csv lido.
Arquivo BasesCenipa\ficheiro SSXM.csv lido.
Arquivo BasesCenipa\ficheiro SSXN.csv lido.
Arquivo BasesCenipa\ficheiro SSXO.csv lido.
Arquivo BasesCenipa\ficheiro SSXP.csv lido.
Arquivo BasesCenipa\ficheiro SSXQ.csv lido.
Arquivo BasesCenipa\ficheiro SSXR.csv lido.
Arquivo BasesCenipa\ficheiro SSXS.csv lido.
Arquivo BasesCenipa\ficheiro SSXT.csv lido.
Arquivo BasesCenipa\ficheiro SSXU.csv lido.
Arquivo BasesCenipa\ficheiro SSXV.csv lido.
Arquivo BasesCenipa\ficheiro SSXW.csv lido.
Arquivo BasesCenipa\ficheiro SSXX.csv lido.
Arquivo BasesCenipa\ficheiro SSXY.csv lido.
Arquivo BasesCenipa\ficheiro SSXZ.csv lido.
Arquivo BasesCenipa\ficheiro SSYA.csv lido.
Arquivo BasesCenipa\ficheiro SSYB.csv lido.
Arquivo BasesCenipa\ficheiro SSY

Arquivo BasesCenipa\ficheiro SWIW.csv lido.
Arquivo BasesCenipa\ficheiro SWIX.csv lido.
Arquivo BasesCenipa\ficheiro SWIY.csv lido.
Arquivo BasesCenipa\ficheiro SWIZ.csv lido.
Arquivo BasesCenipa\ficheiro SWJA.csv lido.
Arquivo BasesCenipa\ficheiro SWJB.csv lido.
Arquivo BasesCenipa\ficheiro SWJC.csv lido.
Arquivo BasesCenipa\ficheiro SWJD.csv lido.
Arquivo BasesCenipa\ficheiro SWJF.csv lido.
Arquivo BasesCenipa\ficheiro SWJH.csv lido.
Arquivo BasesCenipa\ficheiro SWJI.csv lido.
Arquivo BasesCenipa\ficheiro SWJL.csv lido.
Arquivo BasesCenipa\ficheiro SWJM.csv lido.
Arquivo BasesCenipa\ficheiro SWJN.csv lido.
Arquivo BasesCenipa\ficheiro SWJO.csv lido.
Arquivo BasesCenipa\ficheiro SWJP.csv lido.
Arquivo BasesCenipa\ficheiro SWJQ.csv lido.
Arquivo BasesCenipa\ficheiro SWJU.csv lido.
Arquivo BasesCenipa\ficheiro SWJV.csv lido.
Arquivo BasesCenipa\ficheiro SWJW.csv lido.
Arquivo BasesCenipa\ficheiro SWJX.csv lido.
Arquivo BasesCenipa\ficheiro SWJY.csv lido.
Arquivo BasesCenipa\ficheiro SWJ

Arquivo BasesCenipa\ficheiro SWUQ.csv lido.
Arquivo BasesCenipa\ficheiro SWUR.csv lido.
Arquivo BasesCenipa\ficheiro SWUU.csv lido.
Arquivo BasesCenipa\ficheiro SWUV.csv lido.
Arquivo BasesCenipa\ficheiro SWUW.csv lido.
Arquivo BasesCenipa\ficheiro SWUX.csv lido.
Arquivo BasesCenipa\ficheiro SWUZ.csv lido.
Arquivo BasesCenipa\ficheiro SWVB.csv lido.
Arquivo BasesCenipa\ficheiro SWVC.csv lido.
Arquivo BasesCenipa\ficheiro SWVF.csv lido.
Arquivo BasesCenipa\ficheiro SWVI.csv lido.
Arquivo BasesCenipa\ficheiro SWVJ.csv lido.
Arquivo BasesCenipa\ficheiro SWVK.csv lido.
Arquivo BasesCenipa\ficheiro SWVN.csv lido.
Arquivo BasesCenipa\ficheiro SWVR.csv lido.
Arquivo BasesCenipa\ficheiro SWVS.csv lido.
Arquivo BasesCenipa\ficheiro SWVU.csv lido.
Arquivo BasesCenipa\ficheiro SWVV.csv lido.
Arquivo BasesCenipa\ficheiro SWVW.csv lido.
Arquivo BasesCenipa\ficheiro SWVX.csv lido.
Arquivo BasesCenipa\ficheiro SWVY.csv lido.
Arquivo BasesCenipa\ficheiro SWVZ.csv lido.
Arquivo BasesCenipa\ficheiro SWW

Arquivo BasesCenipa\ficheiro_SBSV_2018.csv lido.
Arquivo BasesCenipa\ficheiro_SBSV_2019.csv lido.
Arquivo BasesCenipa\ficheiro_SBSV_2020.csv lido.


In [26]:
cenipa_df=pd.concat(base,ignore_index=True)

In [27]:
cenipa_df.to_csv('BasesCenipa/cenipa_df.csv')